In [1]:
#Notebook for experiments in gathering data

In [1]:
import os
import re
import csv
import requests
import warnings
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm, tqdm_notebook
#warnings.filterwarnings('ignore')

In [71]:
male_directors_path = f"./raw_data/director_lists/wiki/male_directors_wiki_latinized_20190616.csv"
female_directors_path = f"./raw_data/director_lists/wiki/female_directors_wiki_latinized_20190616.csv"
male_directors_df = pd.read_csv(male_directors_path)
female_directors_df = pd.read_csv(female_directors_path)
print(len(male_directors_df), len(female_directors_df))

1772 1036


In [72]:
male_names = male_directors_df.name
female_names = female_directors_df.name

In [77]:
path = f"./raw_data/full_deadline_df.csv"
df = pd.read_csv(path)
df = df.drop(df.columns[0], axis=1)
df.head()

,media,article_url,text_path,category,content_type,topics,title,author,published_at,tags
0,deadline,https://deadline.com/2019/01/4-percent-challen...,deadline/0.txt,film,Article,Film,Universal Commits To Time's Up's #4PercentChal...,Patrick Hipes,2019-01-29 14:07:49,"#4PercentChallenge, Time's Up, Universal"
1,deadline,https://deadline.com/2019/03/jared-harris-jare...,deadline/1.txt,film,Article,Film,Jared Harris Joins Jared Leto In Sony's 'Spide...,Amanda N'Duka,2019-03-04 16:28:50,"Jared Harris, Morbius, Sony"
2,deadline,https://deadline.com/2019/03/sky-tin-star-tim-...,deadline/2.txt,film,Article,"Film, International, TV",Sky Orders Third Season Of 'Tin Star'; 42 Hire...,Andreas Wiseman,2019-03-20 04:09:15,"42, Tim Roth, Tin Star"
3,deadline,https://deadline.com/2019/02/jj-abrams-jedi-st...,deadline/3.txt,film,Article,"Film, Hero Nation",'Star Wars IX': The Last Skywalker Film Wraps ...,Geoff Boucher,2019-02-15 12:56:07,"Hero Nation, JJ Abrams, Lucasfilm, Star Wars"
4,deadline,https://deadline.com/2019/03/june-harding-dies...,deadline/4.txt,film,Article,"Film, TV",June Harding Dies: 'The Trouble With Angels' A...,Bruce Haring,2019-03-29 16:34:36,"Hayley Mills, Ida Lupino, June Harding, Obitua..."


In [106]:
def parse_year(str_date):
    return (int)(str_date[:4])

def valid_date(str_date):
    year = parse_year(str_date)
    if year >= 2008 and year <= 2018:
        return True
    else:
        return False;
    
def find_in(line, names):
    res = []
    for name in names:
        if name in line:
            res.append(name)
    return res    
    
def process(df, male_names, female_names):
    males = dict()
    females = dict()
    with tqdm(desc="rows", total=len(df)) as pbar_outer:    
        for row in df.itertuples():
            # check only required dates
            str_date = getattr(row, 'published_at')
            if not valid_date(str_date):
                continue
            # go through text to find directors
            year = parse_year(str_date)                
            text_path = getattr(row, 'text_path')
            for line in open(f'../modified_dfs/{text_path}'):
                # look up male names
                for name in find_in(line, male_names):
                    if year in males:
                        if name in males[year]:
                            males[year][name].append(text_path)                                 
                        else:
                            males[year][name] = [text_path]
                    else:
                        males[year] = {name : [text_path]}
                # look up female names        
                for name in find_in(line, female_names):
                    if year in females:
                        if name in females[year]:
                            females[year][name].append(text_path)                                                             
                        else:
                            females[year][name] = [text_path]
                    else:
                        females[year] = {name : [text_path]}    
            pbar_outer.update(1)            
    return males, females            

males, females = process(df, male_names, female_names)            

rows:  95%|█████████▌| 48639/51090 [10:48<00:32, 75.00it/s] 


In [110]:
f_out = open('./female_mention_per_year.txt', 'w')
for year, dct in females.items():    
    for name, articles in dct.items():
        res = str(year) + ' ' + name + ' '
        for item in articles:
            res += item + ' '
        res += '\n'   
        f_out.write(res)
f_out.close() 

In [111]:
f_out = open('./male_mention_per_year.txt', 'w')
for year, dct in males.items():    
    for name, articles in dct.items():
        res = str(year) + ' ' + name + ' '
        for item in articles:
            res += item + ' '
        res += '\n'   
        f_out.write(res)
f_out.close()